In [1]:
from __future__ import division

from scipy.signal import convolve2d
import matplotlib.pyplot as plt
import numpy as np
from preprocessing import *
from featuresextraction import *
import cv2
import os
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier,RandomForestClassifier
from scipy.stats import mode

In [2]:
def lpq(img,winSize=3,freqestim=1,mode='nh'):
    rho=0.90

    STFTalpha=1/winSize  # alpha in STFT approaches (for Gaussian derivative alpha=1)
    sigmaS=(winSize-1)/4 # Sigma for STFT Gaussian window (applied if freqestim==2)
    sigmaA=8/(winSize-1) # Sigma for Gaussian derivative quadrature filters (applied if freqestim==3)

    convmode='valid' # Compute descriptor responses only on part that have full neigborhood. Use 'same' if all pixels are included (extrapolates np.image with zeros).

    img=np.float64(img) # Convert np.image to double
    r=(winSize-1)/2 # Get radius from window size
    x=np.arange(-r,r+1)[np.newaxis] # Form spatial coordinates in window

    if freqestim==1:  #  STFT uniform window
        #  Basic STFT filters
        w0=np.ones_like(x)
        w1=np.exp(-2*np.pi*x*STFTalpha*1j)
        w2=np.conj(w1)

    ## Run filters to compute the frequency response in the four points. Store np.real and np.imaginary parts separately
    # Run first filter
    filterResp1=convolve2d(convolve2d(img,w0.T,convmode),w1,convmode)
    filterResp2=convolve2d(convolve2d(img,w1.T,convmode),w0,convmode)
    filterResp3=convolve2d(convolve2d(img,w1.T,convmode),w1,convmode)
    filterResp4=convolve2d(convolve2d(img,w1.T,convmode),w2,convmode)

    # Initilize frequency domain matrix for four frequency coordinates (np.real and np.imaginary parts for each frequency).
    freqResp=np.dstack([filterResp1.real, filterResp1.imag,
                        filterResp2.real, filterResp2.imag,
                        filterResp3.real, filterResp3.imag,
                        filterResp4.real, filterResp4.imag])

    ## Perform quantization and compute LPQ codewords
    inds = np.arange(freqResp.shape[2])[np.newaxis,np.newaxis,:]
    LPQdesc=((freqResp>0)*(2**inds)).sum(2)

    ## Switch format to uint8 if LPQ code np.image is required as output
    if mode=='im':
        LPQdesc=np.uint8(LPQdesc)

    ## Histogram if needed
    if mode=='nh' or mode=='h':
        LPQdesc=np.histogram(LPQdesc.flatten(),range(256))[0]

    ## Normalize histogram if needed
    if mode=='nh':
        LPQdesc=LPQdesc/LPQdesc.sum()

    return LPQdesc

In [3]:
HVSL_feature = []
LVL_features = []
HPP_features = []
TOS_features = []
Stats_features = []
TH_features = []
lpq_features = []
labels = []
feature_to_plot = []
for i in range(1, 10):
    input_dir = f'ACdata_base/{i}/'
    dirs = os.listdir(input_dir)
    for idx,img_dir in enumerate(dirs):
        #print("processing img "+str(img_dir))
        pre = img_dir.split('.')[0]
        img_dir = input_dir + img_dir
        img_rgb = cv2.imread(img_dir)
        img = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
        binarizedImg =  Binarize_Histogram(img,pre)

        ####preprocessing for feature extraction:
        ####our skeletonization
        skeletonized = Skeletonization(binarizedImg, pre)

        ###their skeletonization
        skeleton = 255 - skeletonize(1-binarizedImg/255)*255
        ####edge detection:
        edged = LaplacianEdge(binarizedImg, pre)
        
        LVL_descriptor = getLVL(skeletonized,img,pre)
        HVSL_decriptor = getHVSL(edged,img_rgb,pre) #contains 2 features (V/H,(#of black pixels/((H/total)+(V/total))))
        HPP_descriptor = getHPP(cropToText(binarizedImg),pre)
        TOS_descriptor = getTOS(img)
        Stats_descriptor = getStatsFeatures(binarizedImg)
        lpq_decriptor= lpq(binarizedImg)

        lpq_features.append(lpq_decriptor)
        HVSL_feature.append(HVSL_decriptor)
        LVL_features.append(LVL_descriptor)
        HPP_features.append(HPP_descriptor)
        TOS_features.append(TOS_descriptor)
        Stats_features.append(Stats_descriptor)
        #TH_features.append(TH_descriptor)
        labels.append([i])

#######classifier
labels = np.array(labels)    

E:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:3583: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
E:\Anaconda\lib\site-packages\numpy\core\_methods.py:185: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
E:\Anaconda\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [4]:
x_train, x_test, y_train, y_test = train_test_split(lpq_features, labels, test_size=0.25, random_state=10, stratify=labels)
clf = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(500,250),random_state=1,solver='lbfgs',max_iter=10000)
clf = clf.fit(x_train, y_train)
predicted_lpq = clf.predict_proba(x_test)

E:\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(HVSL_feature, labels, test_size=0.25,random_state=10, stratify=labels)
clf_HVSL = svm.SVC(probability=True)
clf_HVSL.fit(X_train, y_train)
predicted_HVSL = clf_HVSL.predict_proba(X_test)

E:\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(HPP_features, labels, test_size=0.25,random_state=10, stratify=labels)
clf_HPP = svm.SVC(probability=True)
clf_HPP.fit(X_train, y_train)
predicted_HPP = clf_HPP.predict_proba(X_test)

E:\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(Stats_features, labels, test_size=0.25,random_state=10, stratify=labels)
clf_Stats = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(9,18),random_state=1,solver='lbfgs',max_iter=10000,)
clf_Stats.fit(X_train,y_train)
predicted_Stats = clf_Stats.predict_proba(X_test)

E:\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(TOS_features, labels, test_size=0.25,random_state=10, stratify=labels)
clf_TOS = svm.SVC(probability=True) 
clf_TOS.fit(X_train,y_train)
predicted_TOS = clf_TOS.predict_proba(X_test)

E:\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(LVL_features, labels, test_size=0.25,random_state=10, stratify=labels)
clf_LVL = svm.SVC(probability=True)
clf_LVL.fit(X_train,y_train)
predicted_LVL = clf_LVL.predict_proba(X_test)

E:\Anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [62]:
summation = predicted_HPP + predicted_HVSL + predicted_TOS + predicted_Stats + predicted_LVL+ predicted_lpq
predicted = np.argmax(summation, axis=1) + 1

In [63]:
predicted_lpq3 = np.argmax(predicted_lpq, axis=1)+1
# # print(predicted.shape)
# accuracy_test = np.mean(y_test.flatten()==predicted_lpq) * 100
# print(accuracy_test)
predicted_lpq2 = clf.predict(x_test)
print(
    #f"Classification report for classifier {voting_clf}:\n"
    f"{classification_report(y_test, predicted)}\n"
)

              precision    recall  f1-score   support

           1       0.98      1.00      0.99        48
           2       0.98      1.00      0.99        48
           3       0.93      0.93      0.93        45
           4       0.98      0.93      0.96        46
           5       1.00      0.98      0.99        49
           6       1.00      0.98      0.99        45
           7       0.98      1.00      0.99        46
           8       0.98      1.00      0.99        47
           9       1.00      1.00      1.00        48

    accuracy                           0.98       422
   macro avg       0.98      0.98      0.98       422
weighted avg       0.98      0.98      0.98       422




In [10]:
HVSL_test = []
LVL_test = []
HPP_test = []
TOS_test = []
Stats_test = []
lpq_test = []
labels_test = []
for i in range(1, 10):
    input_dir = f'test_set_v2/{i}/'
    dirs = os.listdir(input_dir)
    for idx,img_dir in enumerate(dirs):
        #print("processing img "+str(img_dir))
        pre = img_dir.split('.')[0]
        img_dir = input_dir + img_dir
        img_rgb = cv2.imread(img_dir)
        img = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
        binarizedImg =  Binarize_Histogram(img,pre)

        ####preprocessing for feature extraction:
        ####our skeletonization
        skeletonized = Skeletonization(binarizedImg, pre)

        ###their skeletonization
        skeleton = 255 - skeletonize(1-binarizedImg/255)*255
        ####edge detection:
        edged = LaplacianEdge(binarizedImg, pre)
        
        LVL_descriptor = getLVL(skeletonized,img,pre)
        HVSL_decriptor = getHVSL(edged,img_rgb,pre) #contains 2 features (V/H,(#of black pixels/((H/total)+(V/total))))
        HPP_descriptor = getHPP(cropToText(binarizedImg),pre)
        TOS_descriptor = getTOS(img)
        Stats_descriptor = getStatsFeatures(binarizedImg)
        lpq_decriptor= lpq(binarizedImg)

        lpq_test.append(lpq_decriptor)
        HVSL_test.append(HVSL_decriptor)
        LVL_test.append(LVL_descriptor)
        HPP_test.append(HPP_descriptor)
        TOS_test.append(TOS_descriptor)
        Stats_test.append(Stats_descriptor)
        labels_test.append([i])



#######classifier
labels_test = np.array(labels_test) 

E:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:3583: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
E:\Anaconda\lib\site-packages\numpy\core\_methods.py:185: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
E:\Anaconda\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [12]:
####testing:
predicted_lpq_test = clf.predict_proba(lpq_test)
predicted_HVSL_test = clf_HVSL.predict_proba(HVSL_test)
predicted_HPP_test = clf_HPP.predict_proba(HPP_test)
predicted_Stats_test = clf_Stats.predict_proba(Stats_test)
predicted_TOS_test = clf_TOS.predict_proba(TOS_test)
predicted_LVL_test = clf_LVL.predict_proba(LVL_test)

summation_test =  predicted_lpq_test + predicted_HPP_test + predicted_LVL_test + predicted_TOS_test + predicted_Stats_test
predicted_test = np.argmax(summation_test, axis=1) + 1

print(
    #f"Classification report for classifier {voting_clf}:\n"
    f"{classification_report(labels_test, predicted_test)}\n"
)


              precision    recall  f1-score   support

           1       1.00      0.46      0.63        13
           2       1.00      0.09      0.17        11
           3       0.83      0.45      0.59        11
           4       0.53      0.91      0.67        11
           5       0.50      0.90      0.64        10
           6       0.50      1.00      0.67        10
           7       1.00      0.78      0.88         9
           8       1.00      0.44      0.62         9
           9       0.80      1.00      0.89        12

    accuracy                           0.67        96
   macro avg       0.80      0.67      0.64        96
weighted avg       0.80      0.67      0.64        96


